In [1]:
# Group Members
# Akinfemi Akin-Aluko
# Satbir Gill
# Chandana Prakash

#Notation
#Xij : unit cost of transportation of customer i to dc j

from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
try:

    #Create a model
    m = Model("Project")

     
    #Data (Provided)
    utc_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/unit_tran_cost.txt', header=None))
    dc_cost_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/dc_cost.txt', header=None))
    dc_cap_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/dc_cap.txt', header=None))
    demand_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/demand.txt', header=None))
    
    #Initialize values
    x = utc_values.shape
    numCustomers=x[0]
    numDCs = x[1]
    
    #Create variables
    utc_vars = []    #unit transport cost variables
    dc_cost_vars = []     #distribution centers fixed cost variables
    dc_cap_vars = []
    
    for i in range(1,numCustomers+1):
        
        for j in range(1,numDCs+1):
            v = "x"+str(i)+"_"+str(j)
            v = m.addVar(vtype=GRB.INTEGER, name=v)
            utc_vars.append(v)
            
            if(i==1):
                p = "y"+str(j)
                p = m.addVar(vtype=GRB.BINARY, name=p)
                dc_cost_vars.append(p)
    #Update variables
    m.update()
    
    #Shape arrays to match data shape
    dc_cost_vars = np.array(dc_cost_vars)
    dc_cost_vars = dc_cost_vars.reshape(numDCs,1)
    utc_vars = np.array(utc_vars)
    utc_vars = utc_vars.reshape(numCustomers,numDCs)
    
     
    #Objective Function
    m.setObjective(np.sum(utc_vars*utc_values)+np.sum(dc_cost_vars*dc_cost_values), GRB.MINIMIZE)
    
    #Constraints
    #Customer Demand
    i=-1
    j=-1
    for v in demand_values:
        i+=1
        m.addConstr(np.sum(utc_vars[i])==demand_values[i],"DemandConstr"+str(i+1))
    
    #Distribution Center Capacity Constraints
    for c in dc_cap_values:
        j+=1
        m.addConstr(np.sum(utc_vars[:,j])<=dc_cap_values[j], "CapacityConstr"+str(j+1))
    
    #Optimize model
    m.optimize()
 
    for val in m.getVars():
        if val.x != 0:         #Not to return zero variables
            print val.varName, val.x
        
    print "Objective Value: ", m.objVal
    
    
except GurobiError:
    print 'Error reported. Solution not found'

Optimize a model with 2020 rows, 40020 columns and 80000 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+00, 9e+04]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e+01, 3e+03]
Found heuristic solution: objective 3.18946e+06
Presolve removed 0 rows and 20 columns
Presolve time: 0.10s
Presolved: 2020 rows, 40000 columns, 80000 nonzeros
Variable types: 0 continuous, 40000 integer (0 binary)

Root relaxation: objective 7.980530e+05, 2602 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    798053.00000 798053.000  0.00%     -    0s

Explored 0 nodes (2602 simplex iterations) in 0.22 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 7.980530000000e+05, best bound 7.980530000000e+05, gap 0.0%
x1_12 26.0
x2_14 37.0
x3_3 23.0
x3_15

/home/akinfemi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:53: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
